In [1]:
print("ok")

ok


In [2]:
!pip install openai python-dotenv langchain langchain-openai yfinance phidata

In [5]:
os.environ["SERP_API_KEY"] = os.getenv("SERP_API_KEY")

In [4]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='../.env')

# Access the OPENAI_API_KEY from environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")




In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [8]:
import yfinance as yf
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(temperature=0)

# Define the function to get stock price
def get_stock_price(symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return f"The current stock price of {symbol} is ${todays_data['Close'].iloc[-1]:.2f}"

# Create a tool for the agent to use
tools = [
    Tool(
        name="StockPrice",
        func=get_stock_price,
        description="Useful for getting the stock price of a company. The input should be the stock symbol of the company."
    )
]

# Initialize the agent
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# # Example usage
# query = "What's the current stock price of Microsoft?"
# response = agent.invoke(query)
# print(response)

/var/folders/_y/bj2d0hld0z9b2sgf06km75p00000gn/T/ipykernel_48780/4112696623.py:24: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [9]:
from langchain_core.tools import tool, StructuredTool
from datetime import date

@tool
def get_stock_price(symbol):
    """Use this tool to get the stock price of a company. The input should be the stock symbol of the company."""
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return f"The current stock price of {symbol} is ${todays_data['Close'].iloc[-1]:.2f}"

In [10]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain import hub

tools = [get_stock_price]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Try to answer user query using available tools.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm = ChatOpenAI(model = "gpt-4o-mini")

finance_agent = create_tool_calling_agent(llm, tools, prompt)

finance_agent_executor = AgentExecutor(agent=finance_agent, tools=tools, verbose=True)


In [9]:
from langchain_core.messages import HumanMessage

response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is the stock price of Apple?")]})
response



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'symbol': 'AAPL'}`


The current stock price of AAPL is $244.47The current stock price of Apple (AAPL) is $244.47.

> Finished chain.


{'messages': [HumanMessage(content='What is the stock price of Apple?', additional_kwargs={}, response_metadata={})],
 'output': 'The current stock price of Apple (AAPL) is $244.47.'}

In [10]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is the last dividend date of Apple?")]})
response



> Entering new AgentExecutor chain...
The last dividend date for Apple Inc. is not available through the current tools I have. However, you can check Apple's official investor relations website or financial news sources for the most accurate and up-to-date information regarding dividend dates.

> Finished chain.


{'messages': [HumanMessage(content='What is the last dividend date of Apple?', additional_kwargs={}, response_metadata={})],
 'output': "The last dividend date for Apple Inc. is not available through the current tools I have. However, you can check Apple's official investor relations website or financial news sources for the most accurate and up-to-date information regarding dividend dates."}

In [13]:
from langchain_core.tools import tool, StructuredTool
from datetime import date

@tool
def company_information(ticker: str) -> dict:
    """Use this tool to retrieve company information like address, industry, sector, company officers, business summary, website,
       marketCap, current price, ebitda, total debt, total revenue, debt-to-equity, etc."""
    
    ticker_obj = yf.Ticker(ticker)
    ticker_info = ticker_obj.get_info()

    return ticker_info

@tool
def last_dividend_and_earnings_date(ticker: str) -> dict:
    """
    Use this tool to retrieve company's last dividend date and earnings release dates.
    It does not provide information about historical dividend yields.
    """
    ticker_obj = yf.Ticker(ticker)
    
    return ticker_obj.get_calendar()

@tool
def summary_of_mutual_fund_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top mutual fund holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    mf_holders = ticker_obj.get_mutualfund_holders()
    
    return mf_holders.to_dict(orient="records")

@tool
def summary_of_institutional_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top institutional holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)   
    inst_holders = ticker_obj.get_institutional_holders()
    
    return inst_holders.to_dict(orient="records")

@tool
def stock_grade_updrages_downgrades(ticker: str) -> dict:
    """
    Use this to retrieve grade ratings upgrades and downgrades details of particular stock.
    It'll provide name of firms along with 'To Grade' and 'From Grade' details. Grade date is also provided.
    """
    ticker_obj = yf.Ticker(ticker)
    
    curr_year = date.today().year
    
    upgrades_downgrades = ticker_obj.get_upgrades_downgrades()
    upgrades_downgrades = upgrades_downgrades.loc[upgrades_downgrades.index > f"{curr_year}-01-01"]
    upgrades_downgrades = upgrades_downgrades[upgrades_downgrades["Action"].isin(["up", "down"])]
    
    return upgrades_downgrades.to_dict(orient="records")

@tool
def stock_splits_history(ticker: str) -> dict:
    """
    Use this tool to retrieve company's historical stock splits data.
    """
    ticker_obj = yf.Ticker(ticker)
    hist_splits = ticker_obj.get_splits()
    
    return hist_splits.to_dict()

@tool
def stock_news(ticker: str) -> dict:
    """
    Use this to retrieve latest news articles discussing particular stock ticker.
    """
    ticker_obj = yf.Ticker(ticker)
    
    return ticker_obj.get_news()

In [14]:
tools = [
    company_information,
    last_dividend_and_earnings_date,
    stock_splits_history,
    summary_of_mutual_fund_holders,
    summary_of_institutional_holders, 
    stock_grade_updrages_downgrades,
    stock_news,
    get_stock_price
]


finance_agent = create_tool_calling_agent(llm, tools, prompt)

finance_agent_executor = AgentExecutor(agent=finance_agent, tools=tools, verbose=True)

In [15]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is the last dividend date of Apple?")]})
response



> Entering new AgentExecutor chain...

Invoking: `last_dividend_and_earnings_date` with `{'ticker': 'AAPL'}`


{'Dividend Date': datetime.date(2025, 2, 13), 'Ex-Dividend Date': datetime.date(2025, 2, 10), 'Earnings Date': [datetime.date(2025, 4, 30), datetime.date(2025, 5, 5)], 'Earnings High': 1.67, 'Earnings Low': 1.5, 'Earnings Average': 1.61696, 'Revenue High': 97716000000, 'Revenue Low': 89370000000, 'Revenue Average': 94206162880}The last dividend date for Apple (AAPL) is February 13, 2025. The ex-dividend date is February 10, 2025.

> Finished chain.


{'messages': [HumanMessage(content='What is the last dividend date of Apple?', additional_kwargs={}, response_metadata={})],
 'output': 'The last dividend date for Apple (AAPL) is February 13, 2025. The ex-dividend date is February 10, 2025.'}

In [16]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?")]})
response



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'NVDA'}`


{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU or vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software f

{'messages': [HumanMessage(content='What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?', additional_kwargs={}, response_metadata={})],
 'output': 'For Nvidia Corporation (ticker: NVDA), here are the financial figures you requested:\n\n- **EBITDA**: $72,741,003,264\n- **Total Debt**: $10,225,000,448\n- **Total Revenue**: $113,268,998,144\n- **Debt-to-Equity Ratio**: 15.516\n\nIf you need more information or further assistance, feel free to ask!'}

In [17]:
# Create a dummy portfolio of 5 stocks
# portfolio = {
#     'AAPL': 50,  # Apple
#     'MSFT': 30,  # Microsoft
#     'GOOGL': 20, # Alphabet (Google)
#     'AMZN': 15,  # Amazon
#     'NVDA': 25   # NVIDIA
# }

# Function to analyze a single stock
def analyze_stock(ticker: str, shares: int):
    response = finance_agent_executor.invoke({
        "messages": [
            HumanMessage(content=f"Provide a brief analysis of {ticker} including current price, P/E ratio, and recent performance.")
        ]
    })
    return f"{ticker} ({shares} shares): {response['output']}"

# Function to analyze the entire portfolio
def analyze_portfolio(portfolio: dict):
    analysis = []
    total_value = 0

    for ticker, shares in portfolio.items():
        stock_analysis = analyze_stock(ticker, shares)
        analysis.append(stock_analysis)

        # Get current price to calculate portfolio value
        price_response = finance_agent_executor.invoke({
            "messages": [
                HumanMessage(content=f"What is the current stock price of {ticker}?")
            ]
        })
        try:
            price = float(price_response['output'].split('$')[1].split()[0])
            total_value += price * shares
        except:
            print(f"Could not parse price for {ticker}")

    return analysis, total_value

# # Analyze the portfolio
# print("Analyzing portfolio...")
# stock_analyses, portfolio_value = analyze_portfolio(portfolio)

# # Print the results
# print("\nPortfolio Analysis:")
# for analysis in stock_analyses:
#     print(analysis)
#     print("-" * 50)